<a href="https://colab.research.google.com/github/sangeetsaurabh/PyTorch_Keras_Experiment/blob/master/Text_Number_Prediction/Text_Number_Prediction_single_step_PyTorch_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Predict the next number in the sequence

Given a set of numbers, goal of the model is to predict next number in the sequence. 

For example, model can be given input like - eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve....

Model will predict next number given the one input. Model in this notebook predicts 21st word given 20 words like above (last step prediction).




In [0]:
#### Make sure that the right version of Torch is there
!pip install torchtext==0.6.0
import torchtext
print(torchtext.__version__)

0.6.0


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#drive.flush_and_unmount

In [0]:
#### Setting up the right seed to make Keras result more consistent
import numpy as np
import tensorflow as tf
import random as python_random

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(123)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
python_random.seed(123)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(1234)


In [0]:
#### Setting up path to import important data preparation Python module
import sys
import os
sys.path.append('/content/drive/My Drive/Colab Notebooks/torch_pipe/')

In [0]:
os.getcwd()

'/content'

In [0]:
####Loading up TPU
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.63.181.178:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.63.181.178:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:
#### Using torch utilities to prepare the features. Importing all the important files
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from Util.human_language_modeling import *
from torch.utils.data import DataLoader
import torch.nn.functional as F
import time
import logging

In [0]:
#### Enabling logging
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [0]:
#### Setting up the batch size and length of the sequence
BATCH_SIZE = 64 ## defining the batch size
bptt = 20 ## back propogration through LSTM
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
### A simplie python function to show text given an array of vectors
def show_text(input_vector):
    separator = ' '
    txt = separator.join([vocab.itos[i] for i in input_vector])
    return txt

#### Download the train and validation data

In [0]:
tokenizer = get_tokenizer("spacy")
train_dataset, valid_dataset = HumanNumbers(root='data',bptt=bptt,batch_size=BATCH_SIZE,data_select=('train', 'valid'))
vocab = train_dataset.get_vocab()

INFO:root:Building Vocab based on /content/drive/My Drive/Colab Notebooks/torch_pipe/Human_Numberlearning/Data/train.txt
3641lines [00:00, 36396.41lines/s]

<function tokenizer at 0x7f34df5ad950>


8001lines [00:00, 35296.82lines/s]
INFO:root:Vocab has 34 entries
INFO:root:Creating train data
INFO:root:Creating valid data


51200
51200
torch.Size([51200, 20])
torch.Size([51200, 20])
14080
14080
torch.Size([14080, 20])
torch.Size([14080, 20])


#### Extract the features for Keras/Tensor Flow implementation

In [0]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, GRU, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers

In [0]:
#### Building input features and lables for machine learning models
train_x = train_dataset.input_data.numpy()
train_label = train_dataset.label_data[:,-1].numpy().astype(int)
train_y = tf.keras.utils.to_categorical(train_label, num_classes=len(vocab.itos))

valid_x = valid_dataset.input_data.numpy()
valid_label = valid_dataset.label_data[:,-1].numpy()
valid_y = tf.keras.utils.to_categorical(valid_label, num_classes=len(vocab.itos))

In [0]:
print(train_x.shape)
print (train_y.shape)
print(valid_x.shape)
print(valid_y.shape)

(51200, 20)
(51200, 34)
(14080, 20)
(14080, 34)


##### Create the batch data for Train and Validation set

In [0]:
#### Setting up Keras dataset to feed into machine learning models
BUFFER_SIZE = train_x.shape[0] ## Shuffling the data across entire dataset before building the batch

train_batch = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_batch = train_batch.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_batch = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))
val_batch = val_batch.batch(BATCH_SIZE)

#### Simple DNN to do the prediction

In [0]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Embedding(len(vocab.itos), 100, input_length=train_x.shape[1]),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(len(vocab.itos), activation="softmax"),
        keras.layers.Lambda(lambda x: x[:,-1])
    ]
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 100)           3400      
_________________________________________________________________
dense_6 (Dense)              (None, 20, 64)            6464      
_________________________________________________________________
batch_normalization_3 (Batch (None, 20, 64)            256       
_________________________________________________________________
dense_7 (Dense)              (None, 20, 34)            2210      
_________________________________________________________________
lambda_1 (Lambda)            (None, 34)                0         
Total params: 12,330
Trainable params: 12,202
Non-trainable params: 128
_________________________________________________________________


In [0]:
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])

In [0]:
#history = model.fit(train_x, train_y, epochs=10, batch_size=64, verbose=1,validation_data=(valid_x,valid_y))
history = model.fit(train_batch, epochs=10, verbose=1,validation_data=val_batch)


Epoch 1/10
800/800 [==============================] - 9s 11ms/step - loss: 1.8777 - categorical_accuracy: 0.2337 - val_loss: 3.1220 - val_categorical_accuracy: 0.0844
Epoch 2/10
800/800 [==============================] - 9s 12ms/step - loss: 1.8393 - categorical_accuracy: 0.2380 - val_loss: 3.0730 - val_categorical_accuracy: 0.1884
Epoch 3/10
800/800 [==============================] - 8s 10ms/step - loss: 1.8364 - categorical_accuracy: 0.2397 - val_loss: 3.0727 - val_categorical_accuracy: 0.0844
Epoch 4/10
800/800 [==============================] - 9s 12ms/step - loss: 1.8329 - categorical_accuracy: 0.2371 - val_loss: 3.1444 - val_categorical_accuracy: 0.0909
Epoch 5/10
800/800 [==============================] - 8s 10ms/step - loss: 1.8303 - categorical_accuracy: 0.2397 - val_loss: 3.1492 - val_categorical_accuracy: 0.0839
Epoch 6/10
800/800 [==============================] - 9s 12ms/step - loss: 1.8287 - categorical_accuracy: 0.2405 - val_loss: 3.1893 - val_categorical_accuracy: 0.083

This sequential layer network didn't produce the best results. This was expected as simple DNN is not the right way to predict 21st word given 20 words. Let's customize DNN to take the sequence of 20 words.

#### DNN to do the prediction

Customizing DNN to process one word at a time in a sequence. This is more like a custom RNN.

In [0]:
from tensorflow.keras import layers
from tensorflow import keras

nh = BATCH_SIZE

# Define a Functional model to do a softmax on final dense layer
inputs = keras.Input((nh))
outputs = layers.Dense(len(vocab.itos), activation="softmax")(inputs)
model = keras.Model(inputs, outputs)

class CustomRNN(tf.keras.Model):
    def __init__(self):
        super(CustomRNN, self).__init__()
        self.embedding1 = layers.Embedding(len(vocab.itos), nh, input_length=train_x.shape[1])
        self.projection_1 = layers.Dense(units=64, activation="relu")
        self.batchnormal = layers.BatchNormalization()
        # Our previously-defined Functional model
        self.classifier = model

    def call(self, inputs):
        ### Initialize the weights
        if inputs.shape[0] == None:
          bs = BATCH_SIZE
        else:
          bs = inputs.shape[0]
        h = tf.zeros(shape=(bs, nh))
        ### going in the loop to pick one word at a time
        for t in range(inputs.shape[1]):
            x = inputs[:, t]
            h = h + self.embedding1(x)
            h = self.batchnormal(self.projection_1(h))
        return self.classifier(h)

rnn_model = CustomRNN()
rnn_model.predict(valid_x).shape


(14080, 34)

In [0]:
adam = Adam(lr=0.01)
rnn_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])

In [0]:
#### Train the model
#history = rnn_model.fit(train_x, train_y, epochs=20, batch_size=64, verbose=1,validation_data=(valid_x,valid_y))
history = rnn_model.fit(train_batch, epochs=20, verbose=1,validation_data=val_batch)

Epoch 1/20
800/800 [==============================] - 9s 11ms/step - loss: 1.6653 - categorical_accuracy: 0.4821 - val_loss: 1.9949 - val_categorical_accuracy: 0.4697
Epoch 2/20
800/800 [==============================] - 8s 10ms/step - loss: 1.0610 - categorical_accuracy: 0.6389 - val_loss: 2.1035 - val_categorical_accuracy: 0.4915
Epoch 3/20
800/800 [==============================] - 8s 10ms/step - loss: 0.8751 - categorical_accuracy: 0.7025 - val_loss: 2.0762 - val_categorical_accuracy: 0.5068
Epoch 4/20
800/800 [==============================] - 8s 10ms/step - loss: 0.7418 - categorical_accuracy: 0.7523 - val_loss: 2.2044 - val_categorical_accuracy: 0.5436
Epoch 5/20
800/800 [==============================] - 8s 10ms/step - loss: 0.6657 - categorical_accuracy: 0.7792 - val_loss: 2.1817 - val_categorical_accuracy: 0.5724
Epoch 6/20
800/800 [==============================] - 8s 10ms/step - loss: 0.5939 - categorical_accuracy: 0.8024 - val_loss: 2.3753 - val_categorical_accuracy: 0.555

As can be seen, performance improved quite a bit with the implmentation of SQL model. Let's try RNN, GRU and LSTM models to see what happens.

#### Simple RNN

In [0]:
model = Sequential()
model.add(Embedding(len(vocab.itos), 64, input_length=train_x.shape[1]))
model.add(SimpleRNN(150))
model.add(Dense(len(vocab.itos), activation='softmax'))

### Compile the model
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])

#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(train_x, train_y, epochs=10, batch_size=64, verbose=1,validation_data=(valid_x,valid_y))
#history = model.fit(train_batch, epochs=10, verbose=1,validation_data=val_batch)
#print model.summary()
print(model)


Epoch 1/10
800/800 [==============================] - 14s 17ms/step - loss: 1.4330 - categorical_accuracy: 0.5035 - val_loss: 2.0118 - val_categorical_accuracy: 0.5027
Epoch 2/10
800/800 [==============================] - 12s 15ms/step - loss: 1.3739 - categorical_accuracy: 0.5139 - val_loss: 1.8240 - val_categorical_accuracy: 0.4976
Epoch 3/10
800/800 [==============================] - 11s 14ms/step - loss: 1.5678 - categorical_accuracy: 0.4784 - val_loss: 2.2471 - val_categorical_accuracy: 0.4240
Epoch 4/10
800/800 [==============================] - 11s 14ms/step - loss: 1.5353 - categorical_accuracy: 0.4814 - val_loss: 2.2322 - val_categorical_accuracy: 0.4533
Epoch 5/10
800/800 [==============================] - 13s 16ms/step - loss: 1.4467 - categorical_accuracy: 0.4915 - val_loss: 2.0925 - val_categorical_accuracy: 0.4751
Epoch 6/10
800/800 [==============================] - 10s 13ms/step - loss: 1.3916 - categorical_accuracy: 0.5005 - val_loss: 2.4835 - val_categorical_accuracy:

#### GRU

In [0]:
model = Sequential()
model.add(Embedding(len(vocab.itos), output_dim=64, input_length=train_x.shape[1]))
model.add(GRU(units=150))
model.add(Dense(len(vocab.itos), activation='softmax'))

### Compile the model
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])

#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
#history = model.fit(train_x, train_y, epochs=10, batch_size=64, verbose=1,validation_data=(valid_x,valid_y))
history = model.fit(train_batch, epochs=10, verbose=1,validation_data=val_batch)
print (model.summary())



Epoch 1/10
800/800 [==============================] - 22s 27ms/step - loss: 0.4697 - categorical_accuracy: 0.8517 - val_loss: 1.2467 - val_categorical_accuracy: 0.7963
Epoch 2/10
800/800 [==============================] - 21s 26ms/step - loss: 0.9145 - categorical_accuracy: 0.7364 - val_loss: 1.5323 - val_categorical_accuracy: 0.6738
Epoch 3/10
800/800 [==============================] - 22s 28ms/step - loss: 0.4715 - categorical_accuracy: 0.8524 - val_loss: 1.4716 - val_categorical_accuracy: 0.7714
Epoch 4/10
800/800 [==============================] - 21s 26ms/step - loss: 0.2146 - categorical_accuracy: 0.9395 - val_loss: 1.5443 - val_categorical_accuracy: 0.8058
Epoch 5/10
800/800 [==============================] - 21s 26ms/step - loss: 0.1236 - categorical_accuracy: 0.9682 - val_loss: 1.9054 - val_categorical_accuracy: 0.7994
Epoch 6/10
800/800 [==============================] - 21s 26ms/step - loss: 0.1129 - categorical_accuracy: 0.9688 - val_loss: 1.8607 - val_categorical_accuracy:

In [0]:
#### Let's try the stateful
model = Sequential()
model.add(Embedding(len(vocab.itos), 64, input_length=train_x.shape[1],batch_input_shape=(BATCH_SIZE,train_x.shape[1])))
model.add(GRU(150,stateful=True))
model.add(Dense(len(vocab.itos), activation='softmax'))

### Compile the model
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])

#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
#history = model.fit(train_x, train_y, epochs=10, batch_size=64, verbose=1,validation_data=(valid_x,valid_y))
history = model.fit(train_batch, epochs=10, verbose=1,validation_data=val_batch)
print (model.summary())


Epoch 1/10
800/800 [==============================] - 19s 24ms/step - loss: 0.4722 - categorical_accuracy: 0.8473 - val_loss: 1.6509 - val_categorical_accuracy: 0.8248
Epoch 2/10
800/800 [==============================] - 19s 23ms/step - loss: 0.8802 - categorical_accuracy: 0.7494 - val_loss: 1.3874 - val_categorical_accuracy: 0.6442
Epoch 3/10
800/800 [==============================] - 19s 24ms/step - loss: 0.4198 - categorical_accuracy: 0.8596 - val_loss: 1.6147 - val_categorical_accuracy: 0.7674
Epoch 4/10
800/800 [==============================] - 19s 23ms/step - loss: 0.1937 - categorical_accuracy: 0.9420 - val_loss: 1.6947 - val_categorical_accuracy: 0.7907
Epoch 5/10
800/800 [==============================] - 19s 23ms/step - loss: 0.1356 - categorical_accuracy: 0.9626 - val_loss: 2.4478 - val_categorical_accuracy: 0.7494
Epoch 6/10
800/800 [==============================] - 19s 23ms/step - loss: 0.0761 - categorical_accuracy: 0.9826 - val_loss: 2.0892 - val_categorical_accuracy:

#### LSTM

In [0]:
model = Sequential()
model.add(Embedding(len(vocab.itos), output_dim=64, input_length=train_x.shape[1]))
model.add(LSTM(units=150))
model.add(Dense(len(vocab.itos), activation='softmax'))

### Compile the model
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])

#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
#history = model.fit(train_x, train_y, epochs=10, batch_size=64, verbose=1,validation_data=(valid_x,valid_y))
history = model.fit(train_batch, epochs=10, verbose=1,validation_data=val_batch)
print (model.summary())

Epoch 1/10
800/800 [==============================] - 26s 33ms/step - loss: 1.0249 - categorical_accuracy: 0.6214 - val_loss: 1.5408 - val_categorical_accuracy: 0.7087
Epoch 2/10
800/800 [==============================] - 25s 31ms/step - loss: 0.1175 - categorical_accuracy: 0.9661 - val_loss: 1.2967 - val_categorical_accuracy: 0.7843
Epoch 3/10
800/800 [==============================] - 25s 31ms/step - loss: 0.0498 - categorical_accuracy: 0.9891 - val_loss: 1.2797 - val_categorical_accuracy: 0.7437
Epoch 4/10
800/800 [==============================] - 24s 30ms/step - loss: 0.0207 - categorical_accuracy: 0.9958 - val_loss: 1.1748 - val_categorical_accuracy: 0.7935
Epoch 5/10
800/800 [==============================] - 24s 30ms/step - loss: 0.0359 - categorical_accuracy: 0.9909 - val_loss: 1.2044 - val_categorical_accuracy: 0.7550
Epoch 6/10
800/800 [==============================] - 25s 31ms/step - loss: 0.0190 - categorical_accuracy: 0.9959 - val_loss: 1.3432 - val_categorical_accuracy:

In [0]:
#### Above is the best model. So saving this model for future testing.
model.save('best_lstm')

INFO:tensorflow:Assets written to: best_lstm/assets


INFO:tensorflow:Assets written to: best_lstm/assets


In [0]:
#### Let's try the stateful
model = Sequential()
model.add(Embedding(len(vocab.itos), 64, input_length=train_x.shape[1],batch_input_shape=(BATCH_SIZE,train_x.shape[1])))
model.add(LSTM(150,stateful=True))
model.add(Dense(len(vocab.itos), activation='softmax'))

### Compile the model
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])

#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
#history = model.fit(train_x, train_y, epochs=10, batch_size=64, verbose=1,validation_data=(valid_x,valid_y))
history = model.fit(train_batch, epochs=10, verbose=1,validation_data=val_batch)
print (model.summary())

Epoch 1/10
800/800 [==============================] - 23s 29ms/step - loss: 1.1438 - categorical_accuracy: 0.5646 - val_loss: 2.0719 - val_categorical_accuracy: 0.5531
Epoch 2/10
800/800 [==============================] - 23s 29ms/step - loss: 0.2572 - categorical_accuracy: 0.9006 - val_loss: 1.5539 - val_categorical_accuracy: 0.7589
Epoch 3/10
800/800 [==============================] - 23s 29ms/step - loss: 0.0206 - categorical_accuracy: 0.9964 - val_loss: 1.7346 - val_categorical_accuracy: 0.7104
Epoch 4/10
800/800 [==============================] - 23s 29ms/step - loss: 0.0379 - categorical_accuracy: 0.9903 - val_loss: 1.1986 - val_categorical_accuracy: 0.7124
Epoch 5/10
800/800 [==============================] - 23s 29ms/step - loss: 0.0125 - categorical_accuracy: 0.9973 - val_loss: 1.3515 - val_categorical_accuracy: 0.7268
Epoch 6/10
800/800 [==============================] - 23s 29ms/step - loss: 0.0152 - categorical_accuracy: 0.9964 - val_loss: 1.1302 - val_categorical_accuracy:

#### See the results with best performing model

In [0]:
from tensorflow import keras
model = keras.models.load_model('lstm')

NotFoundError: ignored

In [0]:
validation_results = model.predict(val_batch)

In [0]:
validation_results[0]

array([1.6416882e-06, 1.4358236e-06, 2.5674004e-07, 2.9088228e-06,
       1.1634426e-05, 8.8351721e-01, 8.0385362e-08, 6.3598834e-02,
       1.7600998e-05, 1.9651961e-02, 5.8387586e-06, 2.1095249e-07,
       1.5345472e-08, 2.2925610e-04, 5.8484642e-05, 8.7771984e-03,
       7.8436422e-08, 7.0348750e-07, 6.4347776e-05, 2.2038482e-07,
       1.8173643e-07, 1.3117283e-04, 5.6981326e-06, 1.2573301e-05,
       8.3909426e-06, 2.5287045e-08, 9.2871618e-03, 5.2801005e-07,
       1.3819874e-02, 2.6627617e-06, 4.6225134e-04, 3.2909756e-04,
       5.4047217e-07, 7.8967517e-08], dtype=float32)

In [0]:
vocab.itos[np.argmax(validation_results[0])]

'five'

In [0]:
show_text(valid_x[0])

'xxBOF \n eight thousand one \n eight thousand two \n eight thousand three \n eight thousand four \n eight thousand'